# Using SupercheQ with Cirq SuperstaQ

[![colab link](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SupertechLabs/client-superstaq/blob/main/docs/source/supercheq/SupercheQ_css.ipynb)

[![binder link?](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/SupertechLabs/client-superstaq/HEAD?labpath=docs/source/supercheq/SupercheQ_css.ipynb)

This notebook demonstrates how to use SupercheQ, a novel quantum fingerprinting protocol developed by Infleqtion. SupercheQ is built into the Superstaq server, and can be accessed using cirq-superstaq.

# Imports and API Token

This example tutorial notebook uses `cirq-superstaq`, our Superstaq client for Cirq; you can try it out by running `pip install cirq-superstaq`:

In [1]:
%pip install cirq-superstaq

In [2]:
# Required imports
import matplotlib.pyplot as plt
import numpy as np
import cirq_superstaq as css

# Optional imports
import os  # used if setting a token as an environment variable

To interface Superstaq via Cirq, we must first instantiate a service provider in `cirq-superstaq` with `Service()`. We then supply a Superstaq API token (or key) by either providing the API token as an argument of `css.Service()` or by setting it as an environment variable (see more details here).

In [3]:
# Get cirq superstaq service for Superstaq backend
service = css.Service()

# File Construction

To demonstrate SupercheQ, we construct 32 files (for the sake of simplicitly, we just use lists in this notebook), each of length 5-bits. The lists represent the 32 different possible 5 bit values, so in theory these should all have distinguishable fingerprints.

In [4]:
# demonstrate fingerprinting on all 32 5-bit bitstrings. We'll encode into just 3 qubits.
# fmt: off
files = [
    [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 0, 1, 1],
    [0, 0, 1, 0, 0], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 0, 1, 1, 1],
    [0, 1, 0, 0, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 0], [0, 1, 0, 1, 1],
    [0, 1, 1, 0, 0], [0, 1, 1, 0, 1], [0, 1, 1, 1, 0], [0, 1, 1, 1, 1],
    [1, 0, 0, 0, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0], [1, 0, 0, 1, 1],
    [1, 0, 1, 0, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0], [1, 0, 1, 1, 1],
    [1, 1, 0, 0, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 0], [1, 1, 0, 1, 1],
    [1, 1, 1, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 1, 0], [1, 1, 1, 1, 1],
]
# fmt: on

SupercheQ uses quantum volume models to generate random circuits, using the file information as a seed. Hence, the final state vectors of the circuit act as a fingerprint for the file. SupercheQ allows you to choose the number of qubits as well as circuit depth. To start with, we will use 3 qubits with a depth of 1.

In [5]:
num_qubits = 3
depth = 1
circuits, fidelities = service.supercheq(files, num_qubits, depth)

In [6]:
print(len(circuits))
print(fidelities.shape)

Let's see what a couple of these circuits look like.

### Circuit for file [0, 0, 0, 0, 0]

In [7]:
circuits[0]

### Circuit for file [0, 0, 0, 0, 1]

In [8]:
circuits[1]

On first glance, these circuits look distinct (which is what we expect). However, we can't know for sure until we study the fidelities. The fidelity is calculated as the inner product of the final state vector pairs. For a good fingerprinting protocol, we would want all identical files to have a fidelity of 1, and distinct pairs to have a fidelity as close to 0 as possible.

The code blocks below allow you to visualize all of the fidelities simultaneously.

In [9]:
def plot_heatmap(fidelities):
    plt.figure(dpi=150)
    plt.imshow(fidelities)
    plt.colorbar()


def plot_histogram(fidelities):
    off_diag = np.tril(fidelities, -1)  # lower-right triangle of fidelities matrix
    on_diag = np.diag(fidelities)
    plt.figure(dpi=150)
    plt.hist([off_diag.flatten(), on_diag], label=["Off-Diagonal", "On-Diagonal"])
    plt.legend()

In [10]:
plot_heatmap(fidelities)

The above graph is a heatmap of the 32 different files and their fidelities with each other. It is essentially a visual representation of the fidelity matrix, with shades closer to yellow representing fidelity values close to one, and darker shades closer to blue/violet indication a fidelity close to 0. As expected, we have a yellow diagonal (1s), surrounded by much darker pixels (0s). 

In [11]:
plot_histogram(fidelities)

In [12]:
np.sum(np.isclose(fidelities, 1))

We can look at the breakdown of fidelities using a histogram. As expected, there are 32 fidelities close to 1. 75% of the histogram entries are in the 0.0 bar, and the rest are distributed over the 0.1-0.6 range. This is a successful fingerprinting in that it pretty clearly distinguised identical pairs from separate ones, but we can do better and push the non identical fidelities closer to 0. We can achieve this by increasing the depth of the circuit or using more qubits.

---
### Scaling Depth

Instead of a circuit depth of 1, let's use 3.

In [13]:
num_qubits, depth = (3, 3)
circuits, fidelities = service.supercheq(files, num_qubits, depth)

In [14]:
plot_heatmap(fidelities)

We see a visible improvement in results with a circuit depth of 6. There are fewer bright pixels around the diagonal, and the diagonal line is much clearer.

In [15]:
plot_histogram(fidelities)

Notice that the off-diagonals have moved slightly to the left in the histogram. Almost 80% of the pairs are in the 0 bar.

Let's try out a few more depths to see if we can get any more gains. The below code block calculates the average value of off diagonal elements for circuit dephts from 1 - 15. Better performance would entail an average off diagonal value closer to 0.

In [16]:
# This cell takes a couple of minutes to run.
mean_off_diags = []
for depth in range(1, 16):
    circuits, fidelities = service.supercheq(files, num_qubits, depth)
    off_diag = np.tril(fidelities, -1)
    mean_off_diags.append(np.mean(off_diag))

In [17]:
plt.bar(range(1, 16), mean_off_diags)
plt.xlabel("Circuit Depth")
plt.ylabel("Average Off Diagonal Value of Fidelity Matrix")
plt.title("Scaling Fingerprinting with Circuit Depth");

It seems like for these files, a circuit depth of 2 is sufficient. Deeper circuits don't seem to have significant gains. The average off diagonal fidelity seems to plateau around 0.06.

----
## Scaling number of qubits

We can similarly increase the number of qubits used. Here we use 4 instead of 3.

In [18]:
num_qubits, depth = (4, 1)
circuits, fidelities = service.supercheq(files, num_qubits, depth)

In [19]:
plot_heatmap(fidelities)

Again, we see a performance improvement. The off diagonal elements are all very dark. This is reflected in the histogram below, where about 90% of the off diagonal elements are in the 0.0 bar.

In [20]:
plot_histogram(fidelities)

Let's try out a few more qubit counts to see if we can get any more gains. The below code block calculates the average value of off diagonal elements for circuit dephts from 1 - 8. Better performance would entail an average off diagonal value closer to 0.

In [21]:
mean_off_diags = []
depth = 1
for num_qubits in range(1, 9):
    circuits, fidelities = service.supercheq(files, num_qubits, depth)
    off_diag = np.tril(fidelities, -1)
    mean_off_diags.append(np.mean(off_diag))

In [22]:
plt.bar(range(1, 9), mean_off_diags)
plt.xlabel("Numer of Qubits")
plt.ylabel("Average Off Diagonal Value of Fidelity Matrix")
plt.title("Scaling Fingerprinting with Qubit Count");

It seems like increasing the number of qubits is more powerful than increasing circuit depth for these circuits. The average value of off diagonal terms consistently decrease and tends towards 0. Below is the fidelity heatmap for 8 qubits. All the off diagonal elements are very close to 0 and virtually indististinguishable.

In [23]:
plot_heatmap(fidelities)

## SupercheQ on files

Similar execution can be done with files instead of bitstrings

In [24]:
def encode_files(files):
    """Takes a list of files and encodes them as a list of lists of ones and zeroes"""
    encoded_files = []
    for i in files:
        f = open(i)
        lines = f.read()
        f.close()
        res = "".join(format(ord(j), "08b") for j in lines)
        encoded_file = []
        for k in res:
            encoded_file.append(int(k))
        encoded_files.append(encoded_file)
    return encoded_files

In [25]:
files = [
    "static/file.txt",
    "static/file1.txt",
    "static/file2.txt",
    "static/file3.txt",
    "static/file4.txt",
    "static/file5.txt",
    "static/file6.txt",
    "static/file7.txt",
    "static/file8.txt",
    "static/file9.txt",
    "static/file10.txt",
    "static/file11.txt",
    "static/file12.txt",
    "static/file13.txt",
    "static/file14.txt",
    "static/file15.txt",
    "static/file15.txt",
    "static/file16.txt",
    "static/file17.txt",
    "static/file18.txt",
    "static/file19.txt",
    "static/file20.txt",
]

In [26]:
encoded_files = encode_files(files)

In [27]:
num_qubits = 3
depth = 1
circuits, fidelities = service.supercheq(encoded_files, num_qubits, depth)

In [28]:
print(len(circuits))
print(fidelities.shape)

In [29]:
circuits[0]

In [30]:
circuits[1]

In [31]:
plot_heatmap(fidelities)

In [32]:
plot_histogram(fidelities)